# Setup

In [1]:
import time

import os
import numpy as np

import tensorflow as tf 
from tensorflow import keras
#tf.config.set_per_process_memory_growth(True)

import pickle

import collections
from collections import defaultdict
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, LeakyReLU, Dropout, Flatten, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import BatchNormalization, Embedding, Reshape, Activation
from keras.layers import Concatenate, Conv2DTranspose, multiply, UpSampling2D
from keras.initializers import RandomNormal
from keras.optimizers import Adam
from keras.utils import Progbar
from keras.metrics import *
from keras import backend as K

import cv2

# Define and initialize Generator

In [2]:
def generator(latent_dim = 100, n_classes = 3):
    # Initialize RandomNormal with mean = 0.0 and stddev = 0.02
    # init = RandomNormal(mean = 0.0, stddev = 0.02)
  
    ### Input 1: class label input ###
    
    # Generator take integer class label as input
    label_input = Input(shape = (1,))
    # print(label_input.shape)
    
    # Embedding layer: to convert class label integer to a vector of size 100
    y = Embedding(n_classes, 100)(label_input)
    # print('Embedding Layer: ', y.shape)
    
    # Dense layer with 7 x 7 units: to convert the vector to a 7 x 7 x 1 tensor
    n_nodes = 7 * 7
    y = Dense(n_nodes, kernel_initializer = RandomNormal(mean = 0.0, stddev = 0.02))(y)
    # print('Dense 1: ', y.shape)
    y = Reshape((7, 7 ,1))(y)
    print('reshape(final y shape): ', y.shape)

    ### Input 2: generator noise input ###
    
    # A latent_dim-dimensional vector is sampled from a normal distribution
    # with mean = 0.0 and stddev = 0.02 
    generator_input = Input(shape = (latent_dim,))
    
    # Noise vector is passed through a dense layer with 1024 * 7 * 7 units 
    # to produce a 7 x 7 x 1024 tensor
    n_nodes = 1024 * 7 * 7
    gen = Dense(n_nodes, 
                kernel_initializer = RandomNormal(mean = 0.0, stddev = 0.02))(generator_input)
    gen = Activation('relu')(gen)
    gen = Reshape((7, 7, 1024))(gen)
    print('Generator noise input: ', gen.shape)
    
    ### Concatenate both the inputs ###
    # The output tensors are then concatenated to produce a 7 × 7 × 1025 tensor. 
    merge = Concatenate()([gen, y])
    print('Concatenate(generator noise input and y: ', merge.shape)

    ### Upsampling ###
    # four successive transposed convolutions 
    # to produce tensors with dimensions 14 × 14 × 512, 28 × 28 × 256, 56 × 56 × 128 and 128 × 128 × 3, respectively.
    
    # (None, 7, 7, 1024) --> (None, 14, 14, 512)
    gen = Conv2DTranspose(512, kernel_size = (5, 5), strides = (2, 2), padding = "same", kernel_initializer = RandomNormal(mean = 0.0, stddev = 0.02))(merge)
    gen = BatchNormalization(momentum = 0.9)(gen)
    gen = Activation("relu")(gen)
    print("(None, 7, 7, 1024) -> (None, 14, 14, 512): ", gen.shape)

    # (None, 14, 14, 512)  --> (None, 28, 28, 256)
    gen = Conv2DTranspose(256, kernel_size = (5, 5), strides = (2, 2), padding = "same", kernel_initializer = RandomNormal(mean = 0.0, stddev = 0.02))(gen)
    gen = BatchNormalization(momentum = 0.9)(gen)
    gen = Activation("relu")(gen)
    print('(None, 14, 14, 512) -> (None, 28, 28, 256): ', gen.shape)

    # (None, 28, 28, 256) --> (None, 56, 56, 128)
    gen = Conv2DTranspose(128, kernel_size = (5, 5), strides = (2, 2), padding = "same", kernel_initializer = RandomNormal(mean = 0.0, stddev = 0.02))(gen)
    gen = BatchNormalization(momentum = 0.9)(gen)
    gen = Activation("relu")(gen)
    print('(None, 28, 28, 256) -> (None, 56, 56, 128): ', gen.shape)

    # (None, 56, 56, 128) --> (None, 112, 112, 3)
    gen = Conv2DTranspose(3, kernel_size = (5, 5), strides = (2, 2), padding = "same", kernel_initializer = RandomNormal(mean = 0.0, stddev = 0.02))(gen)
    out_layer = Activation("tanh")(gen)
    print("(None, 56, 56, 128) -> (None, 112, 112, 3): ", out_layer.shape)
    
    # The final output from the generator is an fake image X of dimension 112 × 112 × 3
    model = Model(inputs = [generator_input, label_input], outputs = out_layer)
    return model

In [3]:
generator_instance = generator(latent_dim = 100, n_classes = 3)
generator_instance.summary()

reshape(final y shape):  (None, 7, 7, 1)
Generator noise input:  (None, 7, 7, 1024)
Concatenate(generator noise input and y:  (None, 7, 7, 1025)
(None, 7, 7, 1024) -> (None, 14, 14, 512):  (None, 14, 14, 512)
(None, 14, 14, 512) -> (None, 28, 28, 256):  (None, 28, 28, 256)
(None, 28, 28, 256) -> (None, 56, 56, 128):  (None, 56, 56, 128)
(None, 56, 56, 128) -> (None, 112, 112, 3):  (None, 112, 112, 3)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                      

# Load weights

In [4]:
# Change x with epoch number
epoch = 323
filename = f"./AC-cGAN/best_weights/params_generator_epoch_{epoch}.hdf5"

In [5]:
generator_instance.load_weights(filename)

# Tools functions

In [6]:
# Function to generate a batch of noise and label 

def generate_batch_noise_and_labels(batch_size, latent_dim, gen_class = "normal"):
    name_map = {'COVID-19':0, 'normal':1, 'pneumonia':2}
    # generate a new batch of noise
    noise = np.random.uniform(-1, 1, (batch_size, latent_dim))

    # defined labels
    labels = np.full(batch_size, name_map[gen_class], dtype=int)

    return noise, labels

# Generation

In [7]:
# Define number of images to generate
generate_class_n = {'COVID-19':0, 'normal':8405, 'pneumonia':0}
# generate_class_n = {'COVID-19':8000, 'normal':8000, 'pneumonia':8000}
# Batch size
batch_size = 256
# Path to save images
path_save = f'./AC-cGAN/generated_images/epoch_{epoch}'
# Class map
#class_map = {0:'COVID-19', 1:'normal', 2:'pneumonia'}

In [8]:
with open(f"{path_save}images_list.txt", "w+") as file:
    for key, value in generate_class_n.items():
        if value != 0:
            n_batch = (value // batch_size) + 1 # produce n_batch of images (surplus)
            counter_image = 0
            for i in range(n_batch):
                noise, labels = generate_batch_noise_and_labels(batch_size = batch_size, latent_dim = 100, gen_class = key)
                
                generated_images_batch = generator_instance.predict([noise, labels.reshape((-1, 1))], verbose=0)
                norm_image = cv2.normalize((generated_images_batch + 1) * 127.5, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                norm_img = norm_image.astype(np.uint8)
                for j in range(norm_img.shape[0]):
                    cv2.imwrite(f"{path_save}/{key}/{key}_{counter_image}.png", norm_img[j,:,:])
                    file.write(f"{key}_{counter_image}.png {key}\n")
                    counter_image += 1
                    print(f"{key}: {counter_image}/{value}")
              

normal: 1/8405
normal: 2/8405
normal: 3/8405
normal: 4/8405
normal: 5/8405
normal: 6/8405
normal: 7/8405
normal: 8/8405
normal: 9/8405
normal: 10/8405
normal: 11/8405
normal: 12/8405
normal: 13/8405
normal: 14/8405
normal: 15/8405
normal: 16/8405
normal: 17/8405
normal: 18/8405
normal: 19/8405
normal: 20/8405
normal: 21/8405
normal: 22/8405
normal: 23/8405
normal: 24/8405
normal: 25/8405
normal: 26/8405
normal: 27/8405
normal: 28/8405
normal: 29/8405
normal: 30/8405
normal: 31/8405
normal: 32/8405
normal: 33/8405
normal: 34/8405
normal: 35/8405
normal: 36/8405
normal: 37/8405
normal: 38/8405
normal: 39/8405
normal: 40/8405
normal: 41/8405
normal: 42/8405
normal: 43/8405
normal: 44/8405
normal: 45/8405
normal: 46/8405
normal: 47/8405
normal: 48/8405
normal: 49/8405
normal: 50/8405
normal: 51/8405
normal: 52/8405
normal: 53/8405
normal: 54/8405
normal: 55/8405
normal: 56/8405
normal: 57/8405
normal: 58/8405
normal: 59/8405
normal: 60/8405
normal: 61/8405
normal: 62/8405
normal: 63/8405
n